In [1]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
import os

ModuleNotFoundError: No module named 'matplotlib'

In [ ]:
image = cv2.imread('../input/surface-crack-detection/Negative/00002.jpg')
type(image)

In [ ]:
plt.imshow(image)

In [ ]:
image = cv2.imread('../input/surface-crack-detection/Positive/00003.jpg')
type(image)

In [ ]:
plt.imshow(image)

In [ ]:
image.shape

In [ ]:
path = '../input/surface-crack-detection'

In [ ]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Dropout
from keras.layers import Dense, Flatten, Activation
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
model = Sequential()
model.add(Conv2D(input_shape=(227,227,3),filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(Conv2D(filters=64,kernel_size=(3,3),padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=128, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=256, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(Conv2D(filters=512, kernel_size=(3,3), padding="same", activation="relu"))
model.add(MaxPool2D(pool_size=(2,2),strides=(2,2)))
model.add(Flatten())
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=4096,activation="relu"))
model.add(Dense(units=2, activation="softmax"))
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
datagen = ImageDataGenerator(rescale=1.0/255,validation_split=0.3)

In [ ]:
train_data = datagen.flow_from_directory(path,
                                         target_size=[227,227],
                                         batch_size=64,
                                         class_mode='categorical',
                                         subset='training')
test_data = datagen.flow_from_directory(path,
                                        target_size=[227,227],
                                        batch_size=64,
                                        class_mode='categorical',
                                        subset='validation')
                                       


In [ ]:
history=model.fit_generator(train_data,
                           validation_data=test_data,
                           epochs=15,
                           steps_per_epoch=len(train_data),
                           validation_steps=len(test_data))

In [ ]:
model.save_weights("model.h5")

In [ ]:
print(history.history.keys())

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

In [ ]:
from keras.applications.vgg16 import VGG16
from keras.applications.vgg16 import preprocess_input

from keras.applications import InceptionV3
from keras.applications import ResNet50
from keras.models import Model

## VGG16 Architecture

In [ ]:
vgg = VGG16(include_top=False, weights='imagenet', input_shape=(227,227,3))
vgg.summary()

In [ ]:
for layer in vgg.layers:
    layer.trainable=False

In [ ]:
x = Flatten()(vgg.output)
x = Dense(512,activation='relu')(x)
x = Dense(512,activation='relu')(x)
prediction = Dense(2,activation='sigmoid')(x)
model = Model(inputs=vgg.input, outputs=prediction)
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history=model.fit_generator(train_data,
                           validation_data=test_data,
                           epochs=5,
                           steps_per_epoch=len(train_data),
                           validation_steps=len(test_data))

In [ ]:
model.save_weights("vgg_model.h5")

In [ ]:
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

## Inception Architecture

In [ ]:
inception = InceptionV3(include_top=False, weights='imagenet', input_shape=(227,227,3))
inception.summary()

In [ ]:
for layer in inception.layers:
    layer.trainable=False

In [ ]:
x = Flatten()(inception.output)
x = Dense(512,activation='relu')(x)
x = Dense(512,activation='relu')(x)
prediction = Dense(2,activation='sigmoid')(x)
model = Model(inputs=inception.input, outputs=prediction)
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
hist=model.fit_generator(train_data,
                           validation_data=test_data,
                           epochs=5,
                           steps_per_epoch=len(train_data),
                           validation_steps=len(test_data))

In [ ]:
model.save_weights("inception_model.h5")

In [ ]:
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

## Resnet Architecture

In [ ]:
resnet = ResNet50(include_top=False, weights='imagenet', input_shape=(227,227,3))
resnet.summary()

In [ ]:
for layer in resnet.layers:
    layer.trainable=False

In [ ]:
x = Flatten()(resnet.output)
x = Dense(512,activation='relu')(x)
x = Dense(512,activation='relu')(x)
prediction = Dense(2,activation='sigmoid')(x)
model = Model(inputs=resnet.input, outputs=prediction)
model.summary()

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
history=model.fit_generator(train_data,
                           validation_data=test_data,
                           epochs=5,
                           steps_per_epoch=len(train_data),
                           validation_steps=len(test_data))

In [ ]:
model.save_weights("resnet_model.h5")

In [ ]:
plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()